# Create a classical statistic and machine learning for triathlon analysis database (find the difference)
## The pourpose of this analysis is:
- Find the differente from top 3 athlete and average 3 best perfomance in every discipline
  - There's a difference from top 3 and avg3?
  - The top 3 are the best athlete compared to the avg 3?
- Find the difference between top 3 and other position
  - Top 3 athletes is the best perfoprmer in every discipline?
  - There's a difference from top 3 and other position?
  - What is the pposition in the discipline for the top 3 athletes?
- Find the most influenced part of the race (excluded the transition)
  - Using different model of analysis, the result is the same?
  - using statistic and machine learning, qhat is the most influenced part of the race?
  - What is the second and third part of the race most influenced, and the difference betweet them



In [ ]:
# install Thorch use this only for a coolab or jupyter notebook environment
# !pip install torch


In [ ]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import seaborn as sns
from scipy.stats import ttest_ind
from IPython.display import display, Math
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from scipy.stats import friedmanchisquare
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import itertools
from scipy.stats import ttest_ind


To set the correct gender, uncomment the code below

In [ ]:
# upload dataset (standard female o male) uncomment/comment

#train_data_path = "/content/drive/MyDrive/TRIATHLON ANALYSIS/db_standard_female_final.csv"
train_data_path = "/content/drive/MyDrive/TRIATHLON ANALYSIS/db_standard_male_final.csv"

In [ ]:
train_df = pd.read_csv(train_data_path, low_memory=False) # open file
train_df.dtypes

In [ ]:
train_df.tail()

In [ ]:
train_df.head()

Visualize dataset in a scatter chart to undestand the final position and the time

In [ ]:
# Convert 'event_date' in datetime
train_df['event_date'] = pd.to_datetime(train_df['event_date'])

# sort DataFrame previously for 'event_date' and then for 'position'
df_sorted = train_df.sort_values(by=['event_date', 'position'])

# make a scatter plot
plt.figure(figsize=(15, 6))

years = pd.date_range(start=df_sorted['event_date'].min(),
                      end=df_sorted['event_date'].max(),
                      freq='YS')

plt.xticks(years.to_pydatetime(), years.year, rotation=90)

plt.scatter(df_sorted['event_date'], df_sorted['position'], alpha=0.6)
plt.title('Race Positions Over Time')
plt.xlabel('Event Date')
plt.ylabel('Position')

plt.show()

During the data cleaning in the previous notebook, some race are cutted for the position. a time below 2500 seconds are deleted from the dataset
In this chart we visualize the race with no firs, second or third position
- delete race before 1989/01/01
- delete all the race without the first, second and third position

In [ ]:
# Specify the cutoff date
cutoff_date = '1989-01-01'

# Filter the DataFrame to remove rows before the cutoff date
df_sorted = df_sorted.loc[df_sorted['event_date'] >= cutoff_date]
df_sorted.reset_index(drop=True, inplace=True)
df_sorted

In [ ]:
# Some information
length = len(df_sorted)
unique_race = len(df_sorted["event_date"].unique())
race_date = df_sorted["event_date"].unique().strftime('%Y-%m-%d')

print(f"Total row = {length}")
print(f"Total race is = {unique_race}")
print(f"Race date from {race_date[0]} to {race_date[-1]}")

In [ ]:
# Group by year and event_date, then count the occurrences
df_sorted['date_of_event'] = df_sorted['date_of_event'].astype(int)
date_counts = df_sorted.groupby(['date_of_event', 'event_date']).size().reset_index(name='count')

# Filter to only keep dates that are repeated (i.e., count > 1)
repeated_dates = date_counts[date_counts['count'] > 1]

# Count the number of repeated dates for each year
repeated_dates_per_year = repeated_dates.groupby('date_of_event').size()

# Display the result
print(repeated_dates_per_year)

In [ ]:
# plot data
file_female= "/content/drive/MyDrive/TRIATHLON ANALYSIS/db_standard_female_final.csv"
if train_data_path == file_female:
  sex = "(Female)"
else:
  sex = "(Male)"


plt.figure(figsize=(10, 6))

repeated_dates_per_year.plot(kind='line', marker='.', color="#626262", alpha=0.7)
plt.title(f"Number of Event per Year {sex}")
plt.xlabel('Year')
plt.ylabel('Number of Events')
plt.xticks(ticks=repeated_dates_per_year.index, labels=repeated_dates_per_year.index, rotation=90)
plt.grid(True)

for x, y in zip(repeated_dates_per_year.index, repeated_dates_per_year):
    plt.text(int(x), y, str(y), ha="left", va="bottom", fontsize=11)

plt.tight_layout()
plt.show()


In [ ]:
# make a scatter plot to conferm the cutting date
plt.figure(figsize=(15, 6))

years = pd.date_range(start=df_sorted['event_date'].min(),
                      end=df_sorted['event_date'].max(),
                      freq='YS')

plt.xticks(years.to_pydatetime(), years.year, rotation=90)

plt.scatter(df_sorted['event_date'], df_sorted['position'], alpha=0.3)
plt.title('Race Positions Over Time')
plt.xlabel('Event Date')
plt.ylabel('Position')

plt.show()

In [ ]:

def filter_races_with_positions(df, position_col='position', event_col='event_date'):
    # list of event to keep (position 1, 2, 3 in a list)
    valid_events = []

    # group for 'event_date'
    for date, group in df.groupby(event_col):
        # chech if all the positions are present in the group
        if all(pos in group[position_col].values for pos in [1, 2, 3]):
            valid_events.append(date)

    # Filter the DataFrame to keep only the rows with valid events
    filtered_df = df[df[event_col].isin(valid_events)]

    return filtered_df

# aplly the function on DataFrame
df_filtered = filter_races_with_positions(df_sorted)

# check the number of events
print(f"Numero di eventi originali: {df_sorted['event_date'].nunique()}")
print(f"Numero di eventi dopo il filtro: {df_filtered['event_date'].nunique()}")


In [ ]:
# Plot the filtered data
plt.figure(figsize=(15, 8))

# chart of the filtered DataFrame
plt.scatter(df_filtered['event_date'], df_filtered['position'], color='gray', label='All Positions', alpha=0.5)

# set the x and y axis
years = pd.date_range(start=df_filtered['event_date'].min(),
                      end=df_filtered['event_date'].max(),
                      freq='YS')

plt.xticks(years.to_pydatetime(), years.year, rotation=90)

plt.title(f'Race Positions Over Time (Filtered for Positions 1, 2, 3) {sex}')
plt.xlabel('Event Date')
plt.ylabel('Position')
plt.legend()
plt.grid(True)
plt.show()


To have correct data also in the test dataset, visualize the position and the date for the test dataset

We need to check if the data have not duplicates or null row for train and test dataset

In [ ]:
# Check for duplicates for train dataset
duplicates = train_df.duplicated()
if duplicates.any():
    print("Found duplicates. Removing...")
    # Remove duplicates
    df_final_train = train_df.drop_duplicates()
    print("Duplicates removed.")
else:
    print("No duplicates found.")

# Check for missing data in any row
missing_data = train_df.isnull().any(axis=1)
if missing_data.any():
    print(f"Found rows with missing data. Removing {missing_data.sum()} rows...")
    # Remove rows with missing data
    train_df = train_df.dropna()
    print("Rows with missing data removed.")
else:
    print("No missing data found.")

# Make a standard statistic analysis
The analysis have different pourpose:

1. Find average time of the top 3 athlete
2. Discorver the correlation between race
3. Find the performance of the top 3 athlete
4. There's a correltion if I use teh top 3 final position or user the top 3 position in swmi, bike and run?
5. Using a simple linear regression, can I determine the total time of the race in Paris 2024?



Create 2 dataframe
1. With top 3 athlete (the 1st, 2nd, and 3rd place of the race)
2. with the best 3 time in swim, bike, run and total time

In [ ]:
# function to create a top3 dataframe
def create_top3_structure(df, event_col, swim_col, bike_col, run_col, total_time_col, position_col):
    # filtering fot the top 3 positions for each event
    top3_df = df[df[position_col] <= 3].sort_values(by=[event_col, position_col]).groupby(event_col).head(3)

    # select the columns that you need:
    top3_df = top3_df[[event_col, swim_col, bike_col, run_col, total_time_col]]

    return top3_df

# function to create a avg3 dataframe with the same structure
def create_avg3_structure(df, event_col, swim_col, bike_col, run_col, total_time_col):
    avg3_df = pd.DataFrame(columns=[event_col, swim_col, bike_col, run_col, total_time_col])

    for event in df[event_col].unique():
        event_data = df[df[event_col] == event]

        # take the 3 best times for each discipline
        swim_times = event_data[swim_col].nsmallest(3).values
        bike_times = event_data[bike_col].nsmallest(3).values
        run_times = event_data[run_col].nsmallest(3).values
        total_times = event_data[total_time_col].nsmallest(3).values

        # only if there are 3 values in the list
        if len(swim_times) == 3 and len(bike_times) == 3 and len(run_times) == 3 and len(total_times) == 3:
            event_df = pd.DataFrame({
                event_col: [event] * 3,
                swim_col: swim_times,
                bike_col: bike_times,
                run_col: run_times,
                total_time_col: total_times
            })

            avg3_df = pd.concat([avg3_df, event_df], ignore_index=True)

    return avg3_df

# apply the function
train_df_top3 = create_top3_structure(df_filtered, 'event_date', 'swim_time', 'bike_time', 'run_time', 'total_time', 'position')
train_df_avg3 = create_avg3_structure(df_filtered, 'event_date', 'swim_time', 'bike_time', 'run_time', 'total_time')

# check the length of the top3 and avg3
print(f"Lunghezza top3: {len(train_df_top3)}")
print(f"Lunghezza avg3: {len(train_df_avg3)}")
print(f"Colonne top3: {train_df_top3.columns.tolist()}")
print(f"Colonne avg3: {train_df_avg3.columns.tolist()}")

# show the top3 and avg3 header
print(train_df_top3.head())
print(train_df_avg3.head())


In [ ]:
# check the length of the top3 and avg3
len_top3 = len(train_df_top3)
len_avg3 = len(train_df_avg3)

# compare the lengths
if len_top3 == len_avg3:
    print(f"The two DataSet have the same length: {len_top3} rows.")
else:
    print(f"The dataset have different lengths. top3: {len_top3} rows, avg3: {len_avg3} rows.")


In [ ]:
# big_chart function to use any dataframe passed as an argument

def big_chart(df, df_name): #df = name of the dataframe, df_name = string with the name of the dataframe

    # Convert date to datetime if not already done
    df.loc[:, 'event_date'] = pd.to_datetime(df['event_date'])

    years = pd.date_range(start=df['event_date'].min(),
                          end=df['event_date'].max(),
                          freq='YS')

    # Calculate mean and standard deviation
    mean_swim = df['swim_time'].mean()
    std_swim = df['swim_time'].std() * 2

    mean_bike = df['bike_time'].mean()
    std_bike = df['bike_time'].std() * 2

    mean_run = df['run_time'].mean()
    std_run = df['run_time'].std() * 2

    mean_total = df['total_time'].mean()
    std_total = df['total_time'].std() * 2

    # Make a chart for the data
    plt.figure(figsize=(20, 12))

    plt.subplot(4, 1, 1)
    plt.scatter(df['event_date'], df['swim_time'], label='Swim Time', color='blue', alpha=0.3)
    plt.axhline(mean_swim, color='orange', linestyle='dashed', linewidth=2, label='Mean')
    plt.axhline(mean_swim + std_swim, color='red', linestyle='dotted', linewidth=2, label='Std Dev')
    plt.axhline(mean_swim - std_swim, color='red', linestyle='dotted', linewidth=2)
    plt.xlabel('Date')
    plt.ylabel('Time (seconds)')
    plt.title(f'{df_name} Swim Time {sex}')
    plt.xticks(years.to_pydatetime(), years.year, rotation=45)
    #plt.ylim(0, 2000)
    plt.legend()
    plt.grid(True)

    plt.subplot(4, 1, 2)
    plt.scatter(df['event_date'], df['bike_time'], label='Bike Time', color='green', alpha=0.3)
    plt.axhline(mean_bike, color='orange', linestyle='dashed', linewidth=2, label='Mean')
    plt.axhline(mean_bike + std_bike, color='red', linestyle='dotted', linewidth=2, label='Std Dev')
    plt.axhline(mean_bike - std_bike, color='red', linestyle='dotted', linewidth=2)
    plt.xlabel('Date')
    plt.ylabel('Time (seconds)')
    plt.title(f'{df_name} Bike Time {sex}')
    plt.xticks(years.to_pydatetime(), years.year, rotation=45)
    #plt.ylim(0, 6000)
    plt.legend()
    plt.grid(True)

    plt.subplot(4, 1, 3)
    plt.scatter(df['event_date'], df['run_time'], label='Run Time', color='red', alpha=0.3)
    plt.axhline(mean_run, color='orange', linestyle='dashed', linewidth=2, label='Mean')
    plt.axhline(mean_run + std_run, color='red', linestyle='dotted', linewidth=2, label='Std Dev')
    plt.axhline(mean_run - std_run, color='red', linestyle='dotted', linewidth=2)
    plt.xlabel('Date')
    plt.ylabel('Time (seconds)')
    plt.title(f'{df_name} Run Time {sex}')
    plt.xticks(years.to_pydatetime(), years.year, rotation=45)
    #plt.ylim(0, 3600)
    plt.legend()
    plt.grid(True)

    plt.subplot(4, 1, 4)
    plt.scatter(df['event_date'], df['total_time'], label='Total Time', color='purple', alpha=0.3)
    plt.axhline(mean_total, color='orange', linestyle='dashed', linewidth=2, label='Mean')
    plt.axhline(mean_total + std_total, color='red', linestyle='dotted', linewidth=2, label='Std Dev')
    plt.axhline(mean_total - std_total, color='red', linestyle='dotted', linewidth=2)
    plt.xlabel('Date')
    plt.ylabel('Time (seconds)')
    plt.title(f'{df_name} Total Time {sex}')
    plt.xticks(years.to_pydatetime(), years.year, rotation=45)
    #plt.ylim(3600, 10000)
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Use the big_chart function with different dataframes

In [ ]:
big_chart(train_df_top3, "train_df_top3")

In [ ]:
big_chart(train_df_avg3, "train_df_avg3")

In [ ]:
# calculate the correlation

def calcola_correlazione(df1, df2):
    corr = df1[['swim_time', 'bike_time', 'run_time', 'total_time']].corrwith(df2[['swim_time', 'bike_time', 'run_time', 'total_time']])
    return corr

In [ ]:
# calculate the t test

def calcola_ttest_latex(df1, df2):
    ttest_results = {}
    for col in ['swim_time', 'bike_time', 'run_time', 'total_time']:
        t_stat, p_value = ttest_ind(df1[col], df2[col], equal_var=False)
        ttest_results[col] = {'t_statistic': t_stat, 'p_value': p_value}

        # LaTeX formula
        formula_latex = f"For {col}: \, T = {t_stat:.4f}, \, P = {p_value:.12f}"
        display(Math(formula_latex))

        if p_value < 0.01:
            print("Interpretation: Very Good - There is a highly significant difference between the disciplines.")
        elif 0.01 <= p_value < 0.05:
            print("Interpretation: Good - There is a significant difference between the disciplines.")
        elif 0.05 <= p_value < 0.10:
            print("Interpretation: Moderate - There is a marginal difference between the disciplines.")
        elif 0.10 <= p_value < 0.20:
            print("Interpretation: Poor - There is little to no significant difference between the disciplines.")
        else:
            print("Interpretation: Very Poor - There is no significant difference between the disciplines.")

    return ttest_results


In [ ]:
def allinea_dataframes(df1, df2, on_column='event_date'):
    merged_df = pd.merge(df1, df2, on=on_column, suffixes=('_avg3', '_top3'))
    return merged_df

In [ ]:
#calculate the R^2

def calcola_rquadro_latex(df1, df2):
    merged_df = allinea_dataframes(df1, df2)
    r2_results = {}
    for col in ['swim_time', 'bike_time', 'run_time', 'total_time']:
        r2 = r2_score(merged_df[f'{col}_avg3'], merged_df[f'{col}_top3'])
        r2_results[col] = r2

        # LaTeX formula
        formula_latex = f"R^2 \\text{{ for }} {col} = {r2:.4f}"
        display(Math(formula_latex))

    return r2_results


In [ ]:
# create heatmap of the correlation

def plot_heatmap_correlazione(df1, df2):
    combined_df = pd.concat([df1[['swim_time', 'bike_time', 'run_time', 'total_time']],
                             df2[['swim_time', 'bike_time', 'run_time', 'total_time']]],
                            axis=1)
    combined_df.columns = ['swim_time_avg3', 'bike_time_avg3', 'run_time_avg3', 'total_time_avg3',
                           'swim_time_top3', 'bike_time_top3', 'run_time_top3', 'total_time_top3']
    corr = combined_df.corr()
    sns.heatmap(corr, annot=True, cmap='coolwarm')
    plt.title(f"Heat Map correlation between two dataset {sex}")
    plt.show()


In [ ]:
# make linear regression

def regressione_lineare(df1, df2):
    merged_df = allinea_dataframes(df1, df2)
    results = {}
    for col in ['swim_time', 'bike_time', 'run_time', 'total_time']:
        model = LinearRegression()
        X = merged_df[[f'{col}_avg3']].values.reshape(-1, 1)
        y = merged_df[f'{col}_top3'].values
        model.fit(X, y)
        r2 = model.score(X, y)
        coef = model.coef_[0]
        intercept = model.intercept_
        results[col] = {
            'formula': f"${col}_top3 = {coef:.4f} \cdot {col}_avg3 + {intercept:.4f}$",
            'r2': r2
        }
    return results


In [ ]:
# make polinomial regression

def regressione_polinomiale(df1, df2, degree):
    merged_df = allinea_dataframes(df1, df2)
    results = {}
    for col in ['swim_time', 'bike_time', 'run_time', 'total_time']:
        polynomial_features = PolynomialFeatures(degree=degree)
        model = make_pipeline(polynomial_features, LinearRegression())
        X = merged_df[[f'{col}_avg3']].values.reshape(-1, 1)
        y = merged_df[f'{col}_top3'].values
        model.fit(X, y)
        r2 = model.score(X, y)

        # find the coefficients and intercept of the polynomial model
        coeffs = model.named_steps['linearregression'].coef_
        intercept = model.named_steps['linearregression'].intercept_

        # LaTeX formula
        terms = [f"{coeff:.4f}x^{i}" for i, coeff in enumerate(coeffs)]
        formula = f"${col}_top3 = {intercept:.4f} + " + " + ".join(terms) + "$"

        results[col] = {
            'formula': formula,
            'r2': r2
        }
    return results


In [ ]:
# create heat map

def plot_heatmap_ttest(ttest_results):
    # Create DataFrame for T-statistic and p-values
    t_stat_df = pd.DataFrame({col: [ttest_results[col]['t_statistic']] for col in ttest_results})
    p_value_df = pd.DataFrame({col: [ttest_results[col]['p_value']] for col in ttest_results})

    # Plot heatmap for T-statistic
    plt.figure(figsize=(10, 2))
    sns.heatmap(t_stat_df, annot=True, cmap='coolwarm', cbar=True, fmt=".4f")
    plt.title(f'Heatmap for T-test {sex}')
    plt.show()

    # Plot heatmap for P-values
    plt.figure(figsize=(10, 2))
    sns.heatmap(p_value_df, annot=True, cmap='coolwarm', cbar=True, fmt=".8f")
    plt.title(f'Heatmap for P-values {sex}')
    plt.show()


In [ ]:
# finction to covert coluimns in type number
def ensure_numeric(df, columns):
    for col in columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

columns_to_check = ['swim_time', 'bike_time', 'run_time', 'total_time']

# convert columns in type number
train_df_avg3 = ensure_numeric(train_df_avg3, columns_to_check)
train_df_top3 = ensure_numeric(train_df_top3, columns_to_check)

In [ ]:
ttest_results = calcola_ttest_latex(train_df_avg3, train_df_top3)
plot_heatmap_ttest(ttest_results)

In [ ]:
df1 = train_df_top3
df2 = train_df_avg3

# t-test on total_time
t_stat, p_value = ttest_ind(df1['total_time'], df2['total_time'], equal_var=False)

# result interpretation
print(f"T-test: {t_stat}")
print(f"P-value: {p_value}")

if p_value < 0.05:
    print("The average of 'total_time' between two datasets is significantly different.")
else:
    print("The average of 'total_time' between two dataset is not significantly different.")


In [ ]:
def plot_heatmap_rquadro(r2_results):
    # make a dataframe for R^2
    r2_df = pd.DataFrame(r2_results, index=[0])

    # Plot heatmap for R^2
    plt.figure(figsize=(10, 2))
    sns.heatmap(r2_df, annot=True, cmap='coolwarm', cbar=True, fmt=".4f")
    plt.title(f'Heatmap for R-square {sex}')
    plt.show()

r2_results = calcola_rquadro_latex(train_df_avg3, train_df_top3)
plot_heatmap_rquadro(r2_results)

In [ ]:
# make calculation

merged_df = allinea_dataframes(train_df_avg3, train_df_top3)
corr = calcola_correlazione(train_df_avg3[['swim_time', 'bike_time', 'run_time', 'total_time']],
                            train_df_top3[['swim_time', 'bike_time', 'run_time', 'total_time']],)
ttest_results = calcola_ttest_latex(train_df_avg3[['swim_time', 'bike_time', 'run_time', 'total_time']],
                              train_df_top3[['swim_time', 'bike_time', 'run_time', 'total_time']],)
r2 = calcola_rquadro_latex(train_df_avg3, train_df_top3)
plot_heatmap_correlazione(train_df_avg3, train_df_top3)

In [ ]:
# linear regression and polynomial regression
lin_reg_results = regressione_lineare(train_df_avg3, train_df_top3)

poly_reg_results = regressione_polinomiale(train_df_avg3, train_df_top3, degree=3)

lin_reg_results
poly_reg_results


# Position analysis
The pourpose of this analysis is that the winner of the race sometime had the best time in swim, bike or run.

We want to analyze the difference between finel position and swim, bike and run position

In [ ]:
df_filtered

In [ ]:
df_filtered.dropna()
df_filtered

In [ ]:
def plot_positions_with_regression(df):
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    # chart 1: final position Vs. swim position with confidence interval and a line of regression
    # change the confident interval as your use
    sns.regplot(
        x='position',
        y='swim_position',
        data=df,
        ax=axes[0],
        ci=95,
        marker=".",
        scatter_kws={'color': 'blue', 'alpha': 0.3},
        line_kws={'color': 'black', 'lw': 2},
        scatter=True
    )
    axes[0].set_title(f'Final Position Vs. Swim Position {sex}')
    axes[0].set_xlabel('Final Position')
    axes[0].set_ylabel('Swim Position')

    # chart 2: final position Vs. bike position with confidence interval and a line of regression
    sns.regplot(
        x='position',
        y='bike_position',
        data=df,
        ax=axes[1],
        ci=95,
        marker=".",
        scatter_kws={'color': 'green', 'alpha': 0.3},
        line_kws={'color': 'black', 'lw': 2},
        scatter=True
    )
    axes[1].set_title(f'Final Position Vs. Bike Position {sex}')
    axes[1].set_xlabel('Final Position')
    axes[1].set_ylabel('Bike Position')

    # chart 3: final position Vs. run position with confidence interval and a line of regression
    sns.regplot(
        x='position',
        y='run_position',
        data=df,
        ax=axes[2],
        ci=95,
        marker=".",
        scatter_kws={'color': 'red', 'alpha': 0.3},
        line_kws={'color': 'black', 'lw': 2},
        scatter=True
    )
    axes[2].set_title(f'Final Position Vs. Run Position {sex}')
    axes[2].set_xlabel('Final Position ')
    axes[2].set_ylabel('Run Position')

    plt.tight_layout()
    plt.show()

plot_positions_with_regression(df_filtered)


In [ ]:
def regression(df, degree=[1, 5]):
    results = {}

    # Define the independent variable
    X = df['position'].values.reshape(-1, 1)

    # Define the dependent variables
    y_vars = ['swim_position', 'bike_position', 'run_position']

    # Iterate over each dependent variable
    for y_var in y_vars:
        y = df[y_var].values
        results[y_var] = {}

        # Iterate over the range of degrees
        for deg in range(degree[0], degree[1] + 1):
            # Polynomial transformation for degrees > 1
            if deg > 1:
                poly = PolynomialFeatures(degree=deg)
                X_poly = poly.fit_transform(X)
                model = LinearRegression()
                model.fit(X_poly, y)
                y_pred = model.predict(X_poly)
                intercept = model.intercept_
                coefs = model.coef_
                r_squared = r2_score(y, y_pred)

                # Create the polynomial equation string
                equation = f"{y_var} = {intercept:.4f}"
                terms = [f"{coefs[i]:+.4f}*x^{i}" for i in range(1, len(coefs))]
                equation += " " + " ".join(terms)
            else:
                # Linear regression (deg=1)
                model = LinearRegression()
                model.fit(X, y)
                y_pred = model.predict(X)
                intercept = model.intercept_
                slope = model.coef_[0]
                r_squared = r2_score(y, y_pred)
                equation = f"{y_var} = {intercept:.4f} + {slope:.4f}*x"

            # Store results for this variable and degree
            results[y_var][f"degree_{deg}"] = {
                'equation': equation,
                'r_squared': r_squared
            }

    return results

results = regression(df_filtered, degree=[1, 5])

# Print the results
for y_var, degrees in results.items():
    print(f"Results for {y_var}:")
    for deg, info in degrees.items():
        print(f"  {deg}: {info['equation']} (R^2 = {info['r_squared']:.4f})")
    print("\n")


In [ ]:
def get_winners_positions(df):
    # filtering for winners only
    winners = df[df['position'] == 1]
    return winners[['swim_position', 'bike_position', 'run_position']]

In [ ]:
def descriptive_statistics(winners):
    stats = winners.describe()
    return stats

In [ ]:
def plot_winners_positions(winners):
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=winners)
    plt.title(f"Distribution of the discipline position for winner {sex}")
    plt.xlabel('Discipline')
    plt.ylabel('Position')
    plt.ylim(0, 30)
    plt.yticks(np.arange(1, 30 + 1, 1))
    plt.show()

In [ ]:
def compare_disciplines(winners):
    swim_positions = winners['swim_position']
    bike_positions = winners['bike_position']
    run_positions = winners['run_position']

    # Friedman test to check the position distribution
    stat, p_value = friedmanchisquare(swim_positions, bike_positions, run_positions)

    return stat, p_value


In [ ]:
winners_positions = get_winners_positions(df_filtered)

statistics = descriptive_statistics(winners_positions)
print("Statistiche Descrittive:\n", statistics)

plot_winners_positions(winners_positions)

stat, p_value = compare_disciplines(winners_positions)
print(f"Test di Friedman: stat={stat}, p_value={p_value}")

if p_value < 0.01:
    print("Interpretation: Very Good - There is a highly significant difference between the disciplines.")
elif 0.01 <= p_value < 0.05:
    print("Interpretation: Good - There is a significant difference between the disciplines.")
elif 0.05 <= p_value < 0.10:
    print("Interpretation: Moderate - There is a marginal difference between the disciplines.")
elif 0.10 <= p_value < 0.20:
    print("Interpretation: Poor - There is little to no significant difference between the disciplines.")
else:
    print("Interpretation: Very Poor - There is no significant difference between the disciplines.")


# Interpretation Scale for P-Value
Very Good (Highly Significant Difference)

## p-value: Less than 0.01
- Interpretation: There is a very strong evidence that the rankings across the groups differ. This suggests that the performance in different disciplines (e.g., swimming, cycling, running) has a significant and consistent impact on the outcome.
- Example: "The Friedman test indicates a highly significant difference between the disciplines, suggesting that at least one discipline is consistently more influential in determining winners' positions."
Good (Significant Difference)

## p-value: Between 0.01 and 0.05
- Interpretation: There is strong evidence that the rankings across the groups differ. This suggests that differences between disciplines are likely meaningful and affect the outcome.
- Example: "The test shows a significant difference between the disciplines, implying that performance in at least one discipline likely contributes more to the overall result."
Moderate (Marginal Difference)

## p-value: Between 0.05 and 0.10
- Interpretation: There is some evidence that the rankings across the groups differ, but the difference is not as strong. The influence of different disciplines may be present but less pronounced.
- Example: "There is a marginal difference between disciplines, indicating that their impact on the outcome might be there, but it is not as strong."
Poor (Little to No Difference)

## p-value: Between 0.10 and 0.20
- Interpretation: There is weak evidence for differences between the groups. The performance in different disciplines may have little to no significant effect on the outcome.
- Example: "The results suggest little difference between disciplines, implying that no single discipline has a significant impact on the overall outcome."
Very Poor (No Significant Difference)

## p-value: Greater than 0.20
- Interpretation: There is no significant evidence that the rankings differ between groups. The disciplines likely do not have a meaningful impact on the overall result.
- Example: "The Friedman test shows no significant difference between the disciplines, suggesting that each discipline contributes similarly to the final result."

# Definition of the Friedman Test

The Friedman test is a non-parametric statistical test used to detect differences in treatments across multiple test attempts. It is an alternative to the repeated-measures ANOVA when the assumption of normality is violated. The test is particularly useful when you have three or more related samples, such as measurements taken on the same subjects under different conditions.

### Key Points:

- **Purpose**: The Friedman test evaluates whether there are significant differences between the groups based on ranks.
- **Data Type**: It works with ordinal data or non-normally distributed continuous data.
- **Assumptions**: The test assumes that the data is paired and that there is no interaction between the groups.

### Interpreting the Friedman Test:

- **p-value**: The primary output of the Friedman test is the p-value.
  - If the **p-value** is **less than 0.05**, it suggests that there are statistically significant differences between the groups.
  - If the **p-value** is **greater than 0.05**, it suggests that there are no statistically significant differences between the groups.

- **Chi-Square Statistic (Q)**: The Friedman test also provides a chi-square statistic.
  - The larger the **Q** value, the greater the likelihood that the observed ranks differ significantly from what would be expected if there were no differences between the groups.

### Approximate Values for a "Good" Result:

- **p-value < 0.05**: Indicates a significant difference between groups, which might be considered "good" if you're looking for evidence that the treatments or conditions are not equal.
- **Effect Size (if reported)**: An effect size (e.g., Kendall's W) closer to 1 indicates a stronger effect, meaning the differences between groups are more pronounced.

In summary, a "good" result from the Friedman test, indicating significant differences between your conditions or groups, would typically involve a p-value less than 0.05 and a high chi-square statistic.

In [ ]:
def get_positions_in_range(df, start_position, end_position):
    # Filtra il DataFrame per ottenere solo le posizioni finali all'interno dell'intervallo specificato
    filtered_positions = df[(df['position'] >= start_position) & (df['position'] <= end_position)]
    return filtered_positions[['swim_position', 'bike_position', 'run_position']]

In [ ]:
def descriptive_statistics(positions):
    stats = positions.describe()
    return stats

In [ ]:
def plot_positions_in_range(positions, start_position, end_position):
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=positions)
    plt.title(f'Distribution of the discipline position for final position {start_position}-{end_position} {sex}')
    plt.ylim(0, 40)
    plt.xlabel('Discipline')
    plt.ylabel('Position')
    plt.yticks(np.arange(1, 40 + 1, 1))
    plt.show()

In [ ]:
def compare_disciplines(positions):
    swim_positions = positions['swim_position']
    bike_positions = positions['bike_position']
    run_positions = positions['run_position']

    # Friedman test to campare the discipline
    stat, p_value = friedmanchisquare(swim_positions, bike_positions, run_positions)

    if p_value < 0.01:
        print("Interpretation: Very Good - There is a highly significant difference between the disciplines.")
    elif 0.01 <= p_value < 0.05:
        print("Interpretation: Good - There is a significant difference between the disciplines.")
    elif 0.05 <= p_value < 0.10:
        print("Interpretation: Moderate - There is a marginal difference between the disciplines.")
    elif 0.10 <= p_value < 0.20:
        print("Interpretation: Poor - There is little to no significant difference between the disciplines.")
    else:
        print("Interpretation: Very Poor - There is no significant difference between the disciplines.")

    return stat, p_value


In [ ]:
def generate_descriptive_sentence(metric, value, percentage):
    return f"Position {metric} is {int(value)}, and {percentage:.2f}% of the athlete obtain this position"

def calculate_percentages(positions, value):
    # calculate the percentage of athletes who have that position.
    return (positions == value).mean() * 100

def calculate_above_50th_percentile(positions, median_value):
    # count how many athlete have over 50 percentile
    above_50th = positions[positions > median_value].count()
    total = positions.count()
    percentage = (above_50th / total) * 100
    return above_50th, percentage

def analyze_positions_range(df, start_position, end_position):

    positions_in_range = get_positions_in_range(df, start_position, end_position)

    statistics = descriptive_statistics(positions_in_range)
    print("Descriptive statistic:\n", statistics)

    plot_positions_in_range(positions_in_range, start_position, end_position)

    stat, p_value = compare_disciplines(positions_in_range)
    print(f"Friedman Test: stat={stat}, p_value={p_value}")

    if p_value < 0.01:
        print("Interpretation: Very Good - There is a highly significant difference between the disciplines.")
    elif 0.01 <= p_value < 0.05:
        print("Interpretation: Good - There is a significant difference between the disciplines.")
    elif 0.05 <= p_value < 0.10:
        print("Interpretation: Moderate - There is a marginal difference between the disciplines.")
    elif 0.10 <= p_value < 0.20:
        print("Interpretation: Poor - There is little to no significant difference between the disciplines.")
    else:
        print("Interpretation: Very Poor - There is no significant difference between the disciplines.")

    for col in ['swim_position', 'bike_position', 'run_position']:
        print(f"\nDescriptive {col}:")
        for metric in ['min', '25%', '50%', '75%', 'max', 'mean']:
            value = statistics.at[metric, col]
            if metric == 'mean':
                value = round(value)
            percentage = calculate_percentages(positions_in_range[col], value)
            sentence = generate_descriptive_sentence(metric, value, percentage)
            print(sentence)

        median_value = round(statistics.at['50%', col])
        above_50th, percentage_above_50th = calculate_above_50th_percentile(positions_in_range[col], median_value)
        print(f"The position over 50% have {above_50th} athlete and are {percentage_above_50th:.2f}%")


In [ ]:
# comment/uncomment to select range (valid for each analysis)

#analyze_positions_range(df_filtered, 1, 1)
#analyze_positions_range(df_filtered, 1, 3)
analyze_positions_range(df_filtered, 4, 10)


## We want to understand how many athelte win the race and reach the first position in swim, bike and run

In [ ]:
train_df = train_df.loc[~(train_df == 0).any(axis=1)]
train_df

In [ ]:
# Convert column in int to have correct data
train_df['position'] = train_df['position'].astype(int)
train_df['swim_position'] = train_df['swim_position'].astype(int)
train_df['bike_position'] = train_df['bike_position'].astype(int)
train_df['run_position'] = train_df['run_position'].astype(int)

# filering row for 1 position in swim, bike, run
filtered_df = train_df[(train_df['position'] == 1) &
                       (train_df['swim_position'] == 1) &
                       (train_df['bike_position'] == 1) &
                       (train_df['run_position'] == 1)]

# count number of row for that condition
count = filtered_df.shape[0]

percentage_overall = round((count/unique_race)*100, 2)


print(f"Number of times the athlete who finished 1st overall also finished 1st in swim, bike, and run: {count}\n and this is the {percentage_overall}% of the race considered for {sex}")



In [ ]:
# make interval from 1989 to 2024
full_year_range = pd.Series(range(1989, 2025))

# numebr of event for missing data
events_per_year = filtered_df['event_date'].dt.year.value_counts().reindex(full_year_range, fill_value=0).sort_index()

# chart cration
plt.figure(figsize=(12, 6))

# horizontalline black
plt.hlines(y=0, xmin=1989, xmax=2024, color='black', linewidth=2)

# vertical line
for year, count in events_per_year.items():
    plt.vlines(x=year, ymin=0, ymax=count, color='red', linewidth=2)
    plt.text(year, count + 0.1, str(count), ha='center', va='bottom', fontsize=11, color='black')

# set x and y axis
plt.xticks(full_year_range, rotation=90)
plt.yticks(range(0, events_per_year.max() + 1))
plt.xlabel('Year')
plt.ylabel('Number of races')
plt.title(f'Number of winners with overall swim, bike, run (Female)')
plt.grid(False)

plt.show()


In [ ]:
filtered_df.head(10)

## Determining the most influent part of the race using classic statistic with correlation

In [ ]:
def analyze_influence_statistically(df, start_position, end_position):
    # filetering data by position range
    filtered_df = df[(df['position'] >= start_position) & (df['position'] <= end_position)]

    #  calculate statistics for each discipline (swim, bike, run)
    swim_stats = filtered_df['swim_time'].describe()
    bike_stats = filtered_df['bike_time'].describe()
    run_stats = filtered_df['run_time'].describe()

    # calculate correlation between final position and each discipline (swim, bike, run)
    correlations = filtered_df[['swim_time', 'bike_time', 'run_time', 'position']].corr()['position'][:-1]

    # print result
    print(f"Statistical Analysis from {start_position} to {end_position}:\n")
    print("Descriptive statistic for swim_time:\n", swim_stats)
    print("\nDescriptive statistic for bike_time:\n", bike_stats)
    print("\nDescriptive statistic for run_time:\n", run_stats)
    print("\nCorrelation for the final position:\n", correlations)

    # determine the most influenced discipline
    most_influential_corr = correlations.idxmin()
    print(f"\nThe most influential discipline for this position range is: {most_influential_corr}")


analyze_influence_statistically(df_filtered, 1, 1)
#analyze_influence_statistically(df_filtered, 1, 3)
#analyze_influence_statistically(df_filtered, 4, 10)


# Determining the most influent pert of the race using classic statistic with linear regression

In [ ]:
def analyze_influence_statistically(df, start_position, end_position):
    # Filter the data based on the specified position range
    filtered_df = df[(df['position'] >= start_position) & (df['position'] <= end_position)]

    # Define the independent variables (swim_time, bike_time, run_time) and the dependent variable (position)
    X = filtered_df[['swim_time', 'bike_time', 'run_time']]
    y = filtered_df['position']

    # Fit a linear regression model
    model = LinearRegression()
    model.fit(X, y)

    # Get the coefficients for each discipline
    coefficients = pd.Series(model.coef_, index=X.columns)

    # Print the results
    print(f"Statistical analysis using Linear Regression for final positions between {start_position} and {end_position}:\n")
    print("Linear Regression Coefficients for each discipline:\n", round(coefficients, 7))

    # Determine the most influential discipline
    most_influential_linear = coefficients.idxmin()  # Lower coefficient (more negative) indicates a stronger influence
    print(f"\nThe most influential discipline for this position range is: {most_influential_linear}")


#analyze_influence_statistically(df_filtered, 1, 1)
analyze_influence_statistically(df_filtered, 1, 3)
#analyze_influence_statistically(df_filtered, 4, 10)


# Determining the most influent pert of the race using classic statistic with polynomial regression

In [ ]:
def analyze_influence_statistically(df, start_position, end_position, degree=2):
    # Filter the data based on the specified position range
    filtered_df = df[(df['position'] >= start_position) & (df['position'] <= end_position)]

    # Define the independent variables (swim_time, bike_time, run_time) and the dependent variable (position)
    X = filtered_df[['swim_time', 'bike_time', 'run_time']]
    y = filtered_df['position']

    # Apply polynomial features to the independent variables
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_poly = poly.fit_transform(X)

    # Fit a linear regression model on the polynomial features
    model = LinearRegression()
    model.fit(X_poly, y)

    # Extract the coefficients (for interpretation, use the original feature names)
    feature_names = poly.get_feature_names_out(X.columns)
    coefficients = pd.Series(model.coef_, index=feature_names)

    # Print the results
    print(f"Statistical analysis using Polynomial Regression (degree={degree}) for final positions between {start_position} and {end_position}:\n")
    print("Polynomial Regression Coefficients for each discipline:\n", coefficients)

    # Determine the most influential discipline
    # Sum coefficients for each original discipline feature
    influence = {
        'swim_time': coefficients.filter(like='swim_time').sum(),
        'bike_time': coefficients.filter(like='bike_time').sum(),
        'run_time': coefficients.filter(like='run_time').sum()
    }
    most_influential = min(influence, key=influence.get)

    print(f"\nThe most influential discipline for this position range is: {most_influential}")
    print(f"\nTotal influence: {influence}")


#analyze_influence_statistically(df_filtered, 1, 1, degree=2)
analyze_influence_statistically(df_filtered, 1, 3, degree=2)
#analyze_influence_statistically(df_filtered, 4, 10, degree=2)


# Determining the most influent part of the race using machine learning (sklearn)

In [ ]:
def analyze_influence_ml(df, start_position, end_position):
    # Create a target column: 1 if the final position is within the specified range, otherwise 0
    df['target'] = np.where((df['position'] >= start_position) & (df['position'] <= end_position), 1, 0)

    # Select the features and the target
    X = df[['swim_time', 'bike_time', 'run_time']]
    y = df['target']

    # Split the dataset into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Train the logistic regression model
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)

    # Handle case with only one class in y_test
    labels = np.unique(y_test)
    target_names = ['Other Positions', 'Top Positions'][:len(labels)]

    # Generate the classification report
    report = classification_report(y_test, y_pred, target_names=target_names, labels=labels, output_dict=True)

    # Extract precision, recall, and f1-score for the positive class (Top Positions)
    precision = report[target_names[-1]]['precision'] if len(labels) > 1 else "NaN"
    recall = report[target_names[-1]]['recall'] if len(labels) > 1 else "NaN"
    f1_score = report[target_names[-1]]['f1-score'] if len(labels) > 1 else "NaN"

    # Extract coefficients for each discipline
    coefficients = pd.Series(model.coef_[0], index=X.columns)

    # Determine the most influential discipline
    most_influential = coefficients.idxmin()

    # Return metrics and most influential discipline
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'most_influential': most_influential
    }

#ml_results = analyze_influence_ml(df_filtered, 1, 1)
ml_results = analyze_influence_ml(df_filtered, 1, 3)
#ml_results = analyze_influence_ml(df_filtered, 4, 10)

# Print the results
print(f"Model Accuracy: {ml_results['accuracy']}")
print(f"Model Precision: {ml_results['precision']}")
print(f"Model Recall: {ml_results['recall']}")
print(f"Model F1-Score: {ml_results['f1_score']}")
print(f"Most Influential Discipline: {ml_results['most_influential']}")


# Determining the most influent part of the race using machine learning (tensorflow)

In [ ]:
def analyze_influence_tensorflow(df, start_position, end_position):
    # Create a target column: 1 if the final position is within the specified range, otherwise 0
    df['target'] = np.where((df['position'] >= start_position) & (df['position'] <= end_position), 1, 0)

    # Select the features and the target
    X = df[['swim_time', 'bike_time', 'run_time']].values
    y = df['target'].values

    # Standardize the features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Split the dataset into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Create the TensorFlow model
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)

    # Evaluate the model
    y_pred = (model.predict(X_test) > 0.5).astype("int32")

    accuracy = accuracy_score(y_test, y_pred)

    # Handle case with only one class in y_test
    labels = np.unique(y_test)
    target_names = ['Other Positions', 'Top Positions'][:len(labels)]

    report = classification_report(y_test, y_pred, target_names=target_names, labels=labels, output_dict=True)

    precision = report[target_names[-1]]['precision'] if len(labels) > 1 else "NaN"
    recall = report[target_names[-1]]['recall'] if len(labels) > 1 else "NaN"
    f1_score = report[target_names[-1]]['f1-score'] if len(labels) > 1 else "NaN"

    # Extract feature weights to understand the influence of each discipline
    weights = model.layers[0].get_weights()[0].flatten()
    influence = dict(zip(['swim_time', 'bike_time', 'run_time'], weights))
    most_influential = min(influence, key=influence.get)

    # Print results (optional, can be removed if you only want to return the data)
    print(f"TensorFlow Model Analysis for final positions between {start_position} and {end_position}:\n")
    print(f"Model Accuracy: {accuracy:.4f}")
    print("\nClassification Report:\n", classification_report(y_test, y_pred))
    print(f"Feature Weights: {influence}")
    print(f"\nThe most influential discipline for this position range is: {most_influential}")

    # Return metrics and most influential discipline
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'most_influential': most_influential
    }


#tf_results = analyze_influence_tensorflow(df_filtered, 1, 1)
tf_results = analyze_influence_tensorflow(df_filtered, 1, 3)
#tf_results = analyze_influence_tensorflow(df_filtered, 4, 10)

# Print the results
print(f"Model Accuracy: {tf_results['accuracy']}")
print(f"Model Precision: {tf_results['precision']}")
print(f"Model Recall: {tf_results['recall']}")
print(f"Model F1-Score: {tf_results['f1_score']}")
print(f"Most Influential Discipline: {tf_results['most_influential']}")


### Model Evaluation Metrics

1. **Precision**:
   - **Definition**: Precision measures the proportion of correctly predicted positive instances out of all instances predicted as positive by the model.
   - **Formula**: `Precision = TP / (TP + FP)`
   - **Interpretation**: High precision means that most of the positive predictions made by the model are correct (few false positives).

2. **Recall (Sensitivity or True Positive Rate)**:
   - **Definition**: Recall measures the proportion of actual positives that were correctly identified by the model.
   - **Formula**: `Recall = TP / (TP + FN)`
   - **Interpretation**: High recall indicates that the model is good at identifying most of the actual positives (few false negatives).

3. **F1-Score**:
   - **Definition**: The F1-score is the harmonic mean of precision and recall, providing a balanced measure that considers both false positives and false negatives.
   - **Formula**: `F1-Score = 2 * (Precision * Recall) / (Precision + Recall)`
   - **Interpretation**: The F1-score is useful when it is important to balance precision and recall, especially in situations with imbalanced data.

4. **Support**:
   - **Definition**: Support indicates the total number of occurrences of each class in the test data.
   - **Interpretation**: It shows the distribution of classes in the dataset, helping to understand the basis for calculating the metrics.

5. **Accuracy**:
   - **Definition**: Accuracy measures the proportion of correctly predicted instances out of the total predictions.
   - **Formula**: `Accuracy = (TP + TN) / (TP + TN + FP + FN)`
   - **Interpretation**: Accuracy indicates how correct the model is overall in its predictions.

6. **Macro Avg (Macro Average)**:
   - **Definition**: The macro average calculates the simple average of precision, recall, and F1-score for each class, without considering the support of each class.
   - **Interpretation**: It provides a general measure of the model's effectiveness across all classes, treating each class equally.

7. **Weighted Avg (Weighted Average)**:
   - **Definition**: The weighted average calculates the average of precision, recall, and F1-score for each class, weighted by the support of each class.
   - **Interpretation**: It gives an overall measure that accounts for classes with more examples, better reflecting the model's performance on an imbalanced dataset.

### Discipline Weighting

- **Definition**: Discipline weighting is an assessment of the relative influence of each discipline (`swim_time`, `bike_time`, `run_time`) on the final classification result.

- **Procedure**:
  - In the neural network model created with TensorFlow, the weights associated with the features (`swim_time`, `bike_time`, `run_time`) in the first dense layer indicate how much each feature contributes to the final result.
  - After training the model, the weights are extracted. A more negative weight for a feature suggests that a lower value of that feature (e.g., a shorter time) is associated with a better final result (e.g., a top 3 position).
  - **Conclusion**: The discipline with the most negative weight is considered the most influential in determining a good final result because variations in that discipline have a greater impact on the likelihood of achieving a good position.

# Determining the most influent pert of the race using machine learning (PyTorch)
## These models, when applied to your dataset, can offer deep insights into the factors influencing athletes' final positions, providing a more comprehensive analysis than linear or polynomial regression alone.
- Feedforward Neural Network: Good for capturing general nonlinear relationships.
- Convolutional Neural Network (CNN): Useful for exploring spatial relationships in structured data.
- Recurrent Neural Network (RNN) / LSTM: Best for capturing temporal or sequential dependencies, especially if you have time-series data for the disciplines.


## Pythorch polynomial regression

In [ ]:
class PolynomialRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(PolynomialRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        out = self.linear(x)
        return out

def analyze_influence_pytorch(df, start_position, end_position, degree=2, epochs=1000, lr=0.01):
    # Filter the data based on the specified position range
    filtered_df = df[(df['position'] >= start_position) & (df['position'] <= end_position)]

    # Define the independent variables (swim_time, bike_time, run_time) and the dependent variable (position)
    X = filtered_df[['swim_time', 'bike_time', 'run_time']].values
    y = filtered_df['position'].values.reshape(-1, 1)

    # Apply polynomial features to the independent variables
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_poly = poly.fit_transform(X)

    # Standardize the features
    scaler = StandardScaler()
    X_poly = scaler.fit_transform(X_poly)

    # Convert data to PyTorch tensors
    X_tensor = torch.tensor(X_poly, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.float32)

    # Initialize the model
    input_dim = X_poly.shape[1]
    model = PolynomialRegressionModel(input_dim)

    # Define loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Train the model
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_tensor)
        loss = criterion(outputs, y_tensor)
        loss.backward()
        optimizer.step()

        if (epoch+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

    # Extract the coefficients
    with torch.no_grad():
        weights = model.linear.weight.numpy().flatten()

    # Map the coefficients to the polynomial features
    feature_names = poly.get_feature_names_out(['swim_time', 'bike_time', 'run_time'])
    coefficients = pd.Series(weights, index=feature_names)

    # Print the results
    print(f"\nStatistical analysis using Polynomial Regression (degree={degree}) with PyTorch for final positions between {start_position} and {end_position}:\n")
    print("Polynomial Regression Coefficients for each discipline:\n", coefficients)

    # Determine the most influential discipline
    influence = {
        'swim_time': coefficients.filter(like='swim_time').sum(),
        'bike_time': coefficients.filter(like='bike_time').sum(),
        'run_time': coefficients.filter(like='run_time').sum()
    }
    most_influential = min(influence, key=influence.get)

    print(f"\nThe most influential discipline for this position range is: {most_influential}")
    print(f"\nTotal influence: {influence}")


 #analyze_influence_pytorch(df_filtered, 1, 1, degree=2, epochs=1000, lr=0.01)
analyze_influence_pytorch(df_filtered, 1, 3, degree=2, epochs=1000, lr=0.01)
#analyze_influence_pytorch(df_filtered, 4, 10, degree=2, epochs=1000, lr=0.01)


## Pythorch Feedforward Neural Network
1. Feedforward Neural Network (Fully Connected Network)
- A basic neural network with multiple hidden layers can capture complex relationships between input features (like swim, bike, and run times) and the target variable (final position). This model is more flexible than polynomial regression as it can approximate any continuous function.

In [ ]:
class NeuralNetworkModel(nn.Module):
    def __init__(self, input_dim):
        super(NeuralNetworkModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        out = torch.sigmoid(self.fc3(x))
        return out

def analyze_influence_neural_network(df, start_position, end_position, epochs=1000, lr=0.01):
    filtered_df = df[(df['position'] >= start_position) & (df['position'] <= end_position)]
    X = filtered_df[['swim_time', 'bike_time', 'run_time']].values
    y = (filtered_df['position'] <= end_position).astype(int).values.reshape(-1, 1)

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.float32)

    input_dim = X.shape[1]
    model = NeuralNetworkModel(input_dim)

    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_tensor)
        loss = criterion(outputs, y_tensor)
        loss.backward()
        optimizer.step()

        if (epoch+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

    with torch.no_grad():
        y_pred = (outputs.numpy() > 0.5).astype(int)
        y_true = y_tensor.numpy().astype(int)

    # Handle case with only one class in y_true
    labels = np.unique(y_true)
    target_names = ['Other Positions', 'Top Positions'][:len(labels)]

    accuracy = accuracy_score(y_true, y_pred)
    report = classification_report(y_true, y_pred, target_names=target_names, labels=labels, output_dict=True)

    precision = report[target_names[-1]]['precision'] if len(labels) > 1 else "NaN"
    recall = report[target_names[-1]]['recall'] if len(labels) > 1 else "NaN"
    f1_score = report[target_names[-1]]['f1-score'] if len(labels) > 1 else "NaN"

    with torch.no_grad():
        weights = model.fc1.weight.data.numpy().mean(axis=0)

    influence = {
        'swim_time': weights[0],
        'bike_time': weights[1],
        'run_time': weights[2]
    }
    most_influential = min(influence, key=influence.get)

    # Print results (optional)
    print(f"\nNeural Network Model Analysis for positions between {start_position} and {end_position}:\n")
    print(f"Accuracy: {accuracy:.4f}")
    print("\nClassification Report:\n", classification_report(y_true, y_pred, target_names=target_names))
    print(f"\nThe most influential discipline for this position range is: {most_influential}")

    # Return metrics and most influential discipline
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'most_influential': most_influential
    }


#nn_results = analyze_influence_neural_network(df_filtered, 1, 1, epochs=1000, lr=0.01)
nn_results = analyze_influence_neural_network(df_filtered, 1, 3, epochs=1000, lr=0.01)
#nn_results = analyze_influence_neural_network(df_filtered, 4, 10, epochs=1000, lr=0.01)

# Print the results
print(f"Model Accuracy: {nn_results['accuracy']}")
print(f"Model Precision: {nn_results['precision']}")
print(f"Model Recall: {nn_results['recall']}")
print(f"Model F1-Score: {nn_results['f1_score']}")
print(f"Most Influential Discipline: {nn_results['most_influential']}")


## Pythorch Convolutional Neural Networ
2. Convolutional Neural Network (CNN)
- While CNNs are typically used for image data, they can also be applied to structured data by treating the input features as a one-dimensional "image" and learning spatial hierarchies in the data. This might be useful if there are complex interactions between features.

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, input_dim):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=2)
        self.fc1 = nn.Linear(16 * (input_dim - 1), 32)
        self.fc2 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        out = torch.sigmoid(self.fc2(x))
        return out

def analyze_influence_cnn(df, start_position, end_position, epochs=1000, lr=0.01):
    filtered_df = df[(df['position'] >= start_position) & (df['position'] <= end_position)]
    X = filtered_df[['swim_time', 'bike_time', 'run_time']].values
    y = (filtered_df['position'] <= end_position).astype(int).values.reshape(-1, 1)

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    X_tensor = torch.tensor(X, dtype=torch.float32).unsqueeze(1)  # Add a dimension for Conv1d
    y_tensor = torch.tensor(y, dtype=torch.float32)

    input_dim = X.shape[1]
    model = CNNModel(input_dim)

    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_tensor)
        loss = criterion(outputs, y_tensor)
        loss.backward()
        optimizer.step()

        if (epoch+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

    with torch.no_grad():
        y_pred = (outputs.numpy() > 0.5).astype(int)
        y_true = y_tensor.numpy().astype(int)

    # Handle case with only one class in y_true
    labels = np.unique(y_true)
    target_names = ['Other Positions', 'Top Positions'][:len(labels)]

    accuracy = accuracy_score(y_true, y_pred)
    report = classification_report(y_true, y_pred, target_names=target_names, labels=labels, output_dict=True)

    precision = report[target_names[-1]]['precision'] if len(labels) > 1 else "NaN"
    recall = report[target_names[-1]]['recall'] if len(labels) > 1 else "NaN"
    f1_score = report[target_names[-1]]['f1-score'] if len(labels) > 1 else "NaN"

    with torch.no_grad():
        filters = model.conv1.weight.data.numpy().flatten()

    influence = {
        'swim_time': filters[0],
        'bike_time': filters[1],
        'run_time': filters[2]
    }
    most_influential = min(influence, key=influence.get)

    # Print results (optional)
    print(f"\nCNN Model Analysis for positions between {start_position} and {end_position}:\n")
    print(f"Accuracy: {accuracy:.4f}")
    print("\nClassification Report:\n", classification_report(y_true, y_pred, target_names=target_names))
    print(f"\nThe most influential discipline for this position range is: {most_influential}")

    # Return metrics and most influential discipline
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'most_influential': most_influential
    }


#cnn_results = analyze_influence_cnn(df_filtered, 1, 1, epochs=1000, lr=0.01)
cnn_results = analyze_influence_cnn(df_filtered, 1, 3, epochs=1000, lr=0.01)
#cnn_results = analyze_influence_cnn(df_filtered, 4, 10, epochs=1000, lr=0.01)

# Print the results
print(f"Model Accuracy: {cnn_results['accuracy']}")
print(f"Model Precision: {cnn_results['precision']}")
print(f"Model Recall: {cnn_results['recall']}")
print(f"Model F1-Score: {cnn_results['f1_score']}")
print(f"Most Influential Discipline: {cnn_results['most_influential']}")


## Pythorch Recurrent Neural Network (RNN) / Long Short-Term Memory (LSTM)
3. Recurrent Neural Network (RNN) / Long Short-Term Memory (LSTM)
- If the times for swimming, biking, and running are collected over multiple intervals (e.g., splits within the disciplines), an RNN or LSTM could capture the sequential dependencies between these intervals and how they influence the final position.

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, 32, batch_first=True)
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        h_0 = torch.zeros(1, x.size(0), 32).requires_grad_()
        c_0 = torch.zeros(1, x.size(0), 32).requires_grad_()

        output, (h_out, _) = self.lstm(x, (h_0, c_0))
        h_out = h_out.view(-1, 32)
        out = torch.sigmoid(self.fc(h_out))
        return out

def analyze_influence_lstm(df, start_position, end_position, epochs=1000, lr=0.01):
    filtered_df = df[(df['position'] >= start_position) & (df['position'] <= end_position)]
    X = filtered_df[['swim_time', 'bike_time', 'run_time']].values
    y = (filtered_df['position'] <= end_position).astype(int).values.reshape(-1, 1)

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    X_tensor = torch.tensor(X, dtype=torch.float32).unsqueeze(1)  # Add a dimension for LSTM
    y_tensor = torch.tensor(y, dtype=torch.float32)

    input_dim = X.shape[1]
    model = LSTMModel(input_dim)

    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_tensor)
        loss = criterion(outputs, y_tensor)
        loss.backward()
        optimizer.step()

        if (epoch+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

    with torch.no_grad():
        y_pred = (outputs.numpy() > 0.5).astype(int)
        y_true = y_tensor.numpy().astype(int)

    # Handle case with only one class in y_true
    labels = np.unique(y_true)
    target_names = ['Other Positions', 'Top Positions'][:len(labels)]

    accuracy = accuracy_score(y_true, y_pred)
    report = classification_report(y_true, y_pred, target_names=target_names, labels=labels, output_dict=True)

    precision = report[target_names[-1]]['precision'] if len(labels) > 1.1 else "NaN"
    recall = report[target_names[-1]]['recall'] if len(labels) > 1.1 else "NaN"
    f1_score = report[target_names[-1]]['f1-score'] if len(labels) > 1.1 else "NaN"

    with torch.no_grad():
        weights = model.fc.weight.data.numpy().flatten()

    influence = {
        'swim_time': weights[0],
        'bike_time': weights[1],
        'run_time': weights[2]
    }
    most_influential = min(influence, key=influence.get)

    # Print results (optional)
    print(f"\nLSTM Model Analysis for positions between {start_position} and {end_position}:\n")
    print(f"Accuracy: {accuracy:.4f}")
    print("\nClassification Report:\n", classification_report(y_true, y_pred, target_names=target_names))
    print(f"\nThe most influential discipline for this position range is: {most_influential}")

    # Return metrics and most influential discipline
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'most_influential': most_influential
    }


#lstm_results = analyze_influence_lstm(df_filtered, 1, 1, epochs=1000, lr=0.01)
lstm_results = analyze_influence_lstm(df_filtered, 1, 3, epochs=1000, lr=0.01)
#lstm_results = analyze_influence_lstm(df_filtered, 4, 10, epochs=1000, lr=0.01)

# Print the results
print(f"Model Accuracy: {lstm_results['accuracy']}")
print(f"Model Precision: {lstm_results['precision']}")
print(f"Model Recall: {lstm_results['recall']}")
print(f"Model F1-Score: {lstm_results['f1_score']}")
print(f"Most Influential Discipline: {lstm_results['most_influential']}")
